In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [2]:
month = 'MAR'

In [3]:
# https://data.cdc.gov/NCHS/Indicators-of-Reduced-Access-to-Care-Due-to-the-Co/xb3p-q62w
df = pd.read_csv(f"DATA/{month} - Indicators_of_Reduced_Access_to_Care_Due_to_the_Coronavirus_Pandemic_During_Last_4_Weeks.csv")

In [4]:
df.head()

,Indicator,Group,State,Subgroup,Phase,Time Period,Time Period Label,Time Period Start Date,Time Period End Date,Value,Low CI,High CI,Confidence Interval,Quartile Range
0,"Delayed Medical Care, Last 4 Weeks",National Estimate,United States,United States,1,1,Apr 23 - May 5,04/23/2020 12:00:00 AM,05/05/2020 12:00:00 AM,38.7,38.1,39.4,38.1 - 39.4,NaN
1,"Delayed Medical Care, Last 4 Weeks",By Age,United States,18 - 29 years,1,1,Apr 23 - May 5,04/23/2020 12:00:00 AM,05/05/2020 12:00:00 AM,34.8,32.5,37.2,32.5 - 37.2,NaN
2,"Delayed Medical Care, Last 4 Weeks",By Age,United States,30 - 39 years,1,1,Apr 23 - May 5,04/23/2020 12:00:00 AM,05/05/2020 12:00:00 AM,37.3,35.6,39.0,35.6 - 39.0,NaN
3,"Delayed Medical Care, Last 4 Weeks",By Age,United States,40 - 49 years,1,1,Apr 23 - May 5,04/23/2020 12:00:00 AM,05/05/2020 12:00:00 AM,40.3,39.0,41.7,39.0 - 41.7,NaN
4,"Delayed Medical Care, Last 4 Weeks",By Age,United States,50 - 59 years,1,1,Apr 23 - May 5,04/23/2020 12:00:00 AM,05/05/2020 12:00:00 AM,40.7,39.3,42.2,39.3 - 42.2,NaN


In [5]:
df['Time Period'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int64)

In [6]:
df.Indicator.unique()

array(['Delayed Medical Care, Last 4 Weeks',
       'Did Not Get Needed Care, Last 4 Weeks',
       'Delayed or Did Not Get Care, Last 4 Weeks'], dtype=object)

In [7]:
df.Group.unique()

array(['National Estimate', 'By Age', 'By Gender',
       'By Race/Hispanic ethnicity', 'By Education', 'By State'],
      dtype=object)

In [8]:
df.Subgroup.unique()

array(['United States', '18 - 29 years', '30 - 39 years', '40 - 49 years',
       '50 - 59 years', '60 - 69 years', '70 - 79 years',
       '80 years and above', 'Male', 'Female', 'Hispanic or Latino',
       'Non-Hispanic white, single race',
       'Non-Hispanic black, single race',
       'Non-Hispanic Asian, single race',
       'Non-Hispanic, other races and multiple races',
       'Less than a high school diploma', 'High school diploma or GED',
       "Some college/Associate's degree", "Bachelor's degree or higher",
       'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina

In [9]:
# df = df[['Indicator', 'Subgroup', 'Time Period', 'Time Period Start Date', 'Time Period End Date', 'Value', 'Low CI', 'High CI']]
df = df[df['Indicator'] == 'Delayed or Did Not Get Care, Last 4 Weeks']
# df.head()

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1718 entries, 140 to 5153
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Indicator               1718 non-null   object 
 1   Group                   1718 non-null   object 
 2   State                   1718 non-null   object 
 3   Subgroup                1718 non-null   object 
 4   Phase                   1718 non-null   object 
 5   Time Period             1718 non-null   int64  
 6   Time Period Label       1718 non-null   object 
 7   Time Period Start Date  1718 non-null   object 
 8   Time Period End Date    1718 non-null   object 
 9   Value                   1680 non-null   float64
 10  Low CI                  1680 non-null   float64
 11  High CI                 1680 non-null   float64
 12  Confidence Interval     1680 non-null   object 
 13  Quartile Range          1224 non-null   object 
dtypes: float64(3), int64(1), object(10)
me

In [11]:
df['Time Period Start Date'].iloc[0]

'04/23/2020 12:00:00 AM'

In [12]:
df['Time Period End Date'].iloc[0]

'05/05/2020 12:00:00 AM'

In [13]:
df.loc[:,'Time Period Start Date'] = df['Time Period Start Date'].apply(lambda x: datetime.datetime.strptime(x[0:10], '%m/%d/%Y').date()) 
df.loc[:,'Time Period End Date'] = df['Time Period End Date'].apply(lambda x: datetime.datetime.strptime(x[0:10], '%m/%d/%Y').date()) 

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1718 entries, 140 to 5153
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Indicator               1718 non-null   object 
 1   Group                   1718 non-null   object 
 2   State                   1718 non-null   object 
 3   Subgroup                1718 non-null   object 
 4   Phase                   1718 non-null   object 
 5   Time Period             1718 non-null   int64  
 6   Time Period Label       1718 non-null   object 
 7   Time Period Start Date  1718 non-null   object 
 8   Time Period End Date    1718 non-null   object 
 9   Value                   1680 non-null   float64
 10  Low CI                  1680 non-null   float64
 11  High CI                 1680 non-null   float64
 12  Confidence Interval     1680 non-null   object 
 13  Quartile Range          1224 non-null   object 
dtypes: float64(3), int64(1), object(10)
me

In [14]:
df[df.Value.isnull()].head()

,Indicator,Group,State,Subgroup,Phase,Time Period,Time Period Label,Time Period Start Date,Time Period End Date,Value,Low CI,High CI,Confidence Interval,Quartile Range
2698,"Delayed or Did Not Get Care, Last 4 Weeks",National Estimate,United States,United States,-1,1,July 22 - Aug 18,2020-07-22,2020-08-18,NaN,NaN,NaN,NaN,NaN
2699,"Delayed or Did Not Get Care, Last 4 Weeks",By Age,United States,18 - 29 years,-1,1,July 22 - Aug 18,2020-07-22,2020-08-18,NaN,NaN,NaN,NaN,NaN
2700,"Delayed or Did Not Get Care, Last 4 Weeks",By Age,United States,30 - 39 years,-1,1,July 22 - Aug 18,2020-07-22,2020-08-18,NaN,NaN,NaN,NaN,NaN
2701,"Delayed or Did Not Get Care, Last 4 Weeks",By Age,United States,40 - 49 years,-1,1,July 22 - Aug 18,2020-07-22,2020-08-18,NaN,NaN,NaN,NaN,NaN
2702,"Delayed or Did Not Get Care, Last 4 Weeks",By Age,United States,50 - 59 years,-1,1,July 22 - Aug 18,2020-07-22,2020-08-18,NaN,NaN,NaN,NaN,NaN


In [15]:
df = df[df.Value.notnull()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1680 entries, 140 to 5153
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Indicator               1680 non-null   object 
 1   Group                   1680 non-null   object 
 2   State                   1680 non-null   object 
 3   Subgroup                1680 non-null   object 
 4   Phase                   1680 non-null   object 
 5   Time Period             1680 non-null   int64  
 6   Time Period Label       1680 non-null   object 
 7   Time Period Start Date  1680 non-null   object 
 8   Time Period End Date    1680 non-null   object 
 9   Value                   1680 non-null   float64
 10  Low CI                  1680 non-null   float64
 11  High CI                 1680 non-null   float64
 12  Confidence Interval     1680 non-null   object 
 13  Quartile Range          1224 non-null   object 
dtypes: float64(3), int64(1), object(10)
me

In [16]:
df[df.Phase == '-1'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Indicator               0 non-null      object 
 1   Group                   0 non-null      object 
 2   State                   0 non-null      object 
 3   Subgroup                0 non-null      object 
 4   Phase                   0 non-null      object 
 5   Time Period             0 non-null      int64  
 6   Time Period Label       0 non-null      object 
 7   Time Period Start Date  0 non-null      object 
 8   Time Period End Date    0 non-null      object 
 9   Value                   0 non-null      float64
 10  Low CI                  0 non-null      float64
 11  High CI                 0 non-null      float64
 12  Confidence Interval     0 non-null      object 
 13  Quartile Range          0 non-null      object 
dtypes: float64(3), int64(1), object(10)
memory usage: 0.0+

In [17]:
df[df['Quartile Range'].isnull()]['Phase'].value_counts()

1                      228
2                       95
3 (Oct 28 � Dec 21)     76
3 (Jan 6 � Feb 15)      57
Name: Phase, dtype: int64

In [18]:
df[df.Phase == '-1']['Time Period Start Date'].unique()

array([], dtype=object)

In [19]:
df[df.Phase == '1']['Time Period Start Date'].unique()

array([datetime.date(2020, 4, 23), datetime.date(2020, 5, 7),
       datetime.date(2020, 5, 14), datetime.date(2020, 5, 21),
       datetime.date(2020, 5, 28), datetime.date(2020, 6, 4),
       datetime.date(2020, 6, 11), datetime.date(2020, 6, 18),
       datetime.date(2020, 6, 25), datetime.date(2020, 7, 2),
       datetime.date(2020, 7, 9), datetime.date(2020, 7, 16)],
      dtype=object)

In [20]:
df[df.Phase == '2']['Time Period Start Date'].unique()

array([datetime.date(2020, 8, 19), datetime.date(2020, 9, 2),
       datetime.date(2020, 9, 16), datetime.date(2020, 9, 30),
       datetime.date(2020, 10, 14)], dtype=object)

In [21]:
df[df.Phase == '3 (Oct 28 � Dec 21)']['Time Period Start Date'].unique()

array([datetime.date(2020, 10, 28), datetime.date(2020, 11, 11),
       datetime.date(2020, 11, 25), datetime.date(2020, 12, 9)],
      dtype=object)

In [22]:
df[df.Phase == '3 (Jan 6 � Mar 29)']['Time Period Start Date'].unique()

array([], dtype=object)

In [23]:
df[df.Phase == '3.1']['Time Period Start Date'].unique()

array([], dtype=object)

In [24]:
df = df[df.Phase != '-1']

In [25]:
df.Phase.unique()

array(['1', '2', '3 (Oct 28 � Dec 21)', '3 (Jan 6 � Feb 15)'],
      dtype=object)

In [26]:
df_grouped = df.groupby(['Group','Subgroup', 'Time Period']).mean()
df_grouped

Value  Low CI  High CI
Group             Subgroup      Time Period                        
By Age            18 - 29 years 1             40.8    38.3     43.4
                                2             43.4    39.3     47.6
                                3             41.0    38.9     43.2
                                4             38.5    36.4     40.7
                                5             39.6    37.2     42.1
...                                            ...     ...      ...
National Estimate United States 20            37.1    36.4     37.9
                                21            39.5    38.7     40.2
                                22            35.3    34.6     36.0
                                23            35.6    34.8     36.4
                                24            34.2    33.5     34.8

[1680 rows x 3 columns]

In [27]:
# df_grouped.to_csv('COVID_accessToCare.csv')

In [28]:
df.to_csv(f'DATA/DATAFRAME_{month}_accessToCare.csv',index=False)